In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import dwave_networkx as dnx
import pandas as pd
import numpy as np
import json
from scipy.optimize import curve_fit
from music21 import stream, converter, instrument, tinyNotation, environment, metadata
import tol_colors as tc
us = environment.UserSettings()
#us['musicxmlPath'] = 'C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe'
#us['musescoreDirectPNGPath'] = 'C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe'

from LBDM import calculateStrengths
from plots import plotBoundaryStrength, plotLagrange, plotHistogram, plotEnergyGaps, plotSampleGraph
from scores import composeArrangement
from qubo import createBQM

plt.style.use("thesis.mplstyle")

picklePath = lambda id : f"../Pickles/{id}/{id}_"
numberPath = lambda id, num : f"../Pickles/{id}/{num}/{id}_{num}_"
dataPath = lambda id, num : f"../Data/{id}/{num}/{id}_{num}_"
figPath = "../Figures/"

cset = tc.get_colorset("bright")
cmap = tc.get_colormap("YlOrBr")

### Histogram

In [ ]:
df = pd.read_csv(dataPath("haydn-op1", 3) + "sampleset.csv")

def plotHistogram(sampleset: pd.DataFrame) -> None:
    '''
    Plots the histogram of a sampleset.
    '''

    N, _, patches = plt.hist(sampleset["energy"], bins=100)

    norm = mpl.colors.LogNorm(1, N.max())
    for thisfrac, thispatch in zip(N, patches):
        color = cmap(norm(thisfrac))
        thispatch.set_facecolor(color)

    plt.xlabel("Energy")
    plt.ylabel("Count")

def plotEnergyGaps(df: pd.DataFrame) -> None:   

    sorted = df.sort_values("energy")
    gaps = [sorted["energy"].iloc[i+1] - sorted["energy"].iloc[i] for i in range(len(sorted)-1)]

    gaps = [g for g in gaps if g <= 1]

    N, _, patches = plt.hist(gaps, bins=100)

    norm = mpl.colors.LogNorm(1, N.max())
    for thisfrac, thispatch in zip(N, patches):
        color = cmap(norm(thisfrac))
        thispatch.set_facecolor(color)

    f = lambda x,A,t: A * np.exp(-t*x)
    r = np.linspace(0,1,100)
    (A, t), _ = curve_fit(f, r, N)

    print(A, t)
    #plt.plot(r, f(r, A, t), ms=0)
    
    plt.xlim(0,1)

    labels = plt.gca().get_xticklabels()
    labels[0].set_visible(False)

    plt.xlabel("Energy gap")
    plt.ylabel("Count")

plt.figure(figsize=(8,3))
plt.subplot(1, 2, 1)
plotHistogram(df)
plt.subplot(1, 2, 2)
plotEnergyGaps(df)

plt.tight_layout()
plt.savefig(figPath+"haydn-histograms.svg")

### Phrase extraction

In [ ]:
phrases = pd.read_csv(picklePath("haydn-op1") + "phrases.csv", index_col=[0,1])
print(len(phrases))

phrases = pd.read_csv(picklePath("beethoven-op67") + "phrases.csv", index_col=[0,1])
print(len(phrases))

In [ ]:
score = converter.parse(picklePath("haydn-op1") + "score.musicxml")
part = score.parts[0]
df = calculateStrengths(part, 0.3, (0.33, 0.66))

print(len(df.query("IsBoundary")))

plt.figure(1, figsize=(8,2))
plotBoundaryStrength(df, 0.3, cset[0])
plt.savefig("../Figures/haydn-lbdm.svg")

score = converter.parse(picklePath("beethoven-op67") + "score.musicxml")
part = score.parts[7]
df = calculateStrengths(part, 0.25, (0.33, 0.66))

#plt.figure(2)
#plotBoundaryStrength(df, 0.25)
#plt.savefig("../Figures/beethoven-lbdm.svg")

### Lagrange parameters

In [ ]:
def plotLagrange(df: pd.DataFrame, ax, point: tuple = (1,1)) -> None:
    '''
    Plots a parametric plot of the two variable Lagrange parameters.
    '''

    grouped = (
        df.groupby(["Node multiplier", "Edge multiplier"])
        .mean(numeric_only=True)
        .reset_index()
    )

    grouped["Broken"] = grouped["Overlaps"] + grouped["Duplicates"]

    x = grouped["Node multiplier"].unique()
    y = grouped["Edge multiplier"].unique()

    X, Y = np.meshgrid(x, y)
    Z = grouped.pivot(index="Edge multiplier", columns="Node multiplier", values="Broken").values

    ax.plot_surface(X, Y, Z, cmap=cmap, edgecolor='black', lw=0.2, alpha=1, zorder=2.5)
    ax.grid(False)
    ax.xaxis.pane.set_edgecolor('black')
    ax.yaxis.pane.set_edgecolor('black')
    ax.zaxis.pane.set_edgecolor('black')
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    ax.view_init(elev=15, azim=45)

    ax.set(xlim=(0, x.max()), ylim=(0, y.max()), zlim=(0, Z.max()),
       xlabel="Vertex multiplier", ylabel="Edge multiplier",
       xticks=range(2, x.max()+1, 2), yticks=range(2, y.max()+1, 2))

    #ax.contourf(X, Y, Z, zdir='x', offset=0, cmap=cmap, zorder=2)
    #ax.contourf(X, Y, Z, zdir='y', offset=0, cmap=cmap, zorder=2)
    
    # Mark chosen value
    pointx, pointy = point
    pointz = grouped.loc[(grouped["Node multiplier"] == pointx) & (grouped["Edge multiplier"] == pointy), "Broken"].values[0]
    ax.scatter(pointx, pointy, pointz, color="k", s=100, marker="x", zorder=5) 

haydn = pd.read_csv(dataPath("haydn-op1", 3) + "lagrange.csv")
beethoven = pd.read_csv(dataPath("beethoven-op67", 4) + "lagrange.csv")

fig, ax = plt.subplots(1, 2, figsize=(8,4), subplot_kw={"projection": "3d", "computed_zorder": False}) 
ax[0].set_zlabel("Broken constraints")
ax[0].set_title("Haydn Op. 1", loc="left", y=0.82)
ax[1].set_title("Beethoven Op. 67", loc="left", y=0.82)

plotLagrange(haydn, ax[0], (6, 6))
plotLagrange(beethoven, ax[1], (6, 12))
plt.tight_layout()

plt.savefig("../Figures/lagrange-parameters.svg")

### Solver configuration

In [ ]:
def plotChainStrengthAndAnnealTime(id: str, num: int, ax1, ax2, color1="tab:blue", color2="tab:red"):

    data = pd.read_csv(dataPath(id, num) + "chain-strength.csv")

    def meanAndStderr(df: pd.DataFrame, xaxis: str, yaxis: str):
        x = df[xaxis].unique()
        queries = [data.query(f"`{xaxis}` == {i}")[yaxis] for i in x]

        mean = np.array([np.mean(q) for q in queries])
        stderr = np.array([np.std(q)/np.sqrt(len(q)) for q in queries])

        return x, mean, stderr

    # Chain strength

    # Lowest energy
    x, mean, stderr = meanAndStderr(data, "Chain strength", "Lowest energy")
    ax1.set_xlabel("Chain strength")
    ax1.errorbar(x, mean, yerr=stderr, fmt='o', color=color1)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.set_xlim(10,50)

    ax12 = ax1.twinx()
    #x, dupl, stderr = meanAndStderr(data, "Chain strength", "Duplicates")
    #x, ovlp, stderr = meanAndStderr(data, "Chain strength", "Overlaps")
    #mean = dupl + ovlp
    # Chain break fraction
    x, mean, stderr = meanAndStderr(data, "Chain strength", "Chain break fraction")
    ax12.errorbar(x, mean, yerr=stderr, fmt='^', color=color2)
    ax12.tick_params(axis='y', labelcolor=color2)
    ax12.set(ylim=(0, mean.max()))

    labels = ax12.get_yticklabels()
    labels[0].set_visible(False)

    # Anneal time

    data = pd.read_csv(dataPath(id, num) + "anneal-time.csv")

    x, mean, stderr = meanAndStderr(data, "Anneal time", "Lowest energy")
    ax2.errorbar(x, mean, yerr=stderr, fmt='o', color=color1)
    ax2.set(xlabel="Anneal time (μs)", xlim=(0,300))

    return ax12

fig, (row1, row2) = plt.subplots(nrows=2, ncols=2, sharex="row", figsize=(8,5))

plotChainStrengthAndAnnealTime("haydn-op1", 3, row1[0], row2[0], color1=cset[0], color2=cset[1])
ax12 = plotChainStrengthAndAnnealTime("beethoven-op67", 4, row1[1], row2[1], color1=cset[0], color2=cset[1])

row1[0].set_ylabel("Lowest energy", color=cset[0])
row2[0].set_ylabel("Lowest energy")
ax12.set_ylabel("Chain break fraction", color=cset[1])

row1[0].set_title("Haydn Op. 1", loc="left")
row1[1].set_title("Beethoven Op. 67", loc="left")

fig.tight_layout()

#plt.savefig("../Figures/solver-config.svg")

### Compare to classical

In [ ]:
def plotCSV(filepath: str, xaxis: str, yaxis: str, axis, colour: str = "tab:blue", label: str = "", marker: str = "o", ls: str = "--") -> None:

    data = pd.read_csv(filepath)
    x = data[xaxis].unique()

    means = []
    stderr = []

    for r in x:
        query = data.query(f"`{xaxis}` == {r}")[yaxis]
        means.append(np.mean(query))
        stderr.append(np.std(query)/np.sqrt(len(query)))

    f = lambda x,m,c: m*x + c
    popt, _ = curve_fit(f, x, means, sigma=stderr)
    print(popt)
    axis.errorbar(x, means, yerr=stderr, fmt=marker, label=label, c=colour)
    axis.plot(x, f(x,popt[0],popt[1]), ls=ls, c=colour, ms=0, label=label)

    return popt[0], popt[1]


def plotCompareClassical(yaxis: str):

    fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(8,5))

    fp = dataPath("haydn-op1",3)
    m1, c1 = plotCSV(fp+"qa-reads.csv", "Total reads", yaxis, ax1, cset[0], "QA", "o", "--")
    plotCSV(fp+"sd-reads.csv", "Total reads", yaxis, ax1, cset[1], "SD", "s", "-.")
    m2, c2 = plotCSV(fp+"sa-reads.csv", "Total reads", yaxis, ax1, cset[2], "SA", "^", ":")
    ax1.set(ylabel=yaxis)
    ax1.set_title("Haydn Op. 1", loc="left")

    print(f"Haydn Op. 1 would show supremacy at {(c2-c1)/(m1-m2)} reads")

    fp = dataPath("beethoven-op67",3)
    m1, c1 = plotCSV(fp+"qa-reads.csv", "Total reads", yaxis, ax2, cset[0], "QA", "o", "--")
    plotCSV(fp+"sd-reads.csv", "Total reads", yaxis, ax2, cset[1], "SD", "s", "-.")
    m2, c2 = plotCSV(fp+"sa-reads.csv", "Total reads", yaxis, ax2, cset[2], "SA", "^", ":")
    ax2.set(xlabel="Total reads", ylabel=yaxis, xlim=(200,2000))
    ax2.set_title("Beethoven Op. 67", loc="left")

    print(f"Beethoven Op. 67 would show supremacy at {(c2-c1)/(m1-m2)} reads")

    lines, labels = ax1.get_legend_handles_labels()

    # Finally, the legend (that maybe you'll customize differently)
    fig.legend(zip(lines[:3],lines[3:]), labels[:3], title="Solver",
               loc="center right", ncols=3, bbox_to_anchor=(.9, 0.515))
    fig.tight_layout()

    return fig

#plotCompareClassical("Lowest energy").savefig("../Figures/compare-reads.svg")
#plotCompareClassical("Total entropy").savefig("../Figures/compare-entropy.svg")


### Scaling

In [ ]:
def plotCompareScaling(id: str, insts: int, yaxis: str, reads: int, ax, scale: float = 1) -> None:

    instRange = np.arange(insts)

    qa = np.empty_like(instRange)
    qaerr = np.empty_like(instRange)
    sd = np.empty_like(instRange)
    sderr = np.empty_like(instRange)
    sa = np.empty_like(instRange)
    saerr = np.empty_like(instRange)

    for i in instRange:

        fp = dataPath(id, i+1)
        data = pd.read_csv(fp+"qa-reads.csv")
        query = data.query(f"`Total reads` == {reads}")[yaxis]
        qa[i] = np.mean(query)
        qaerr[i] = np.std(query)/np.sqrt(len(query))

        data = pd.read_csv(fp+"sd-reads.csv")
        query = data.query(f"`Total reads` == {reads}")[yaxis]
        sd[i] = np.mean(query) - qa[i]
        sderr[i] = np.std(query)/np.sqrt(len(query))

        data = pd.read_csv(fp+"sa-reads.csv")
        query = data.query(f"`Total reads` == {reads}")[yaxis]
        sa[i] = np.mean(query) - qa[i]
        saerr[i] = np.std(query)/np.sqrt(len(query))

    #plt.errorbar(insts+1, qa, yerr=qaerr, label="QA", marker="o")
    ax.errorbar(instRange+1, sd*scale, yerr=sderr*scale, label="SD", fmt="s", color=cset[1])
    ax.errorbar(instRange+1, sa*scale, yerr=saerr*scale, label="SA", fmt="^", color=cset[2])
    ax.hlines(0, 1, 4, linestyles=":", colors="black")
    ax.set(xlim=(1,4), xticks=instRange+1)

In [ ]:
fig, (row1, row2) = plt.subplots(nrows=2, ncols=2, sharex=True, sharey="row",figsize=(8,4))

row1[0].set(ylabel="Energy difference", ylim=(-300,200), yticks=range(-300,200,150))

plotCompareScaling("haydn-op1", 4, "Lowest energy", 2000, row1[0])
plotCompareScaling("beethoven-op67", 4, "Lowest energy", 2000, row1[1])

row1[0].set_title("Haydn Op. 1", loc="left")
row1[1].set_title("Beethoven Op. 67", loc="left")

row2[0].set(ylabel="Time difference (s)", ylim=(-5,20), xlabel="Instruments")
row2[1].set(xlabel="Instruments")

plotCompareScaling("haydn-op1", 4, "Time", 2000, row2[0], scale=1e-6)
plotCompareScaling("beethoven-op67", 4, "Time", 2000, row2[1], scale=1e-6)

lines, labels = row1[0].get_legend_handles_labels()
#fig.legend(lines, labels, loc="center", bbox_to_anchor=(.535, .515), ncols=2, title="Solver")

fig.tight_layout()

#plt.savefig("../Figures/scaling.svg")

### Embedding

In [ ]:
def plotEmbedding(id: str, insts: int, ax, colour: str, marker: str, title: str, extend = 0):

    phrases = pd.read_csv(picklePath(id) + "phrases.csv", index_col=[0,1])

    embedRange = np.arange(insts) + 1
    instRange = np.arange(insts + extend) + 1
    bqms = np.array([len(phrases)*(i+1) for i in instRange])
    bqms[0] = len(phrases)

    ax.set(xlabel="Instruments", xlim=(instRange[0], instRange[-1]), xticks=instRange, ylim=(0,7500))
    ax.plot(instRange, bqms, c=colour, ms=0, label="Logical")
    ax.set_title(title, loc="left")

    embeddings = np.empty(insts)
    
    for i in embedRange:
        embedding = json.load(open(numberPath(id, i) + "embedding.json"))
        embeddings[i-1] = sum(len(j) for j in embedding.values())

    ax.scatter(embedRange, embeddings, c=colour, marker=marker, label="Embedding")
    ax.hlines(5640, 1, insts+extend, linestyles=":", colors="k")
    
    if extend != 0:
        ax.axvspan(insts, insts+extend, color="gray", alpha=.3)

    f = lambda x,a,b,c: a*x**2 + b*x + c
    (a, b, c), _ = curve_fit(f, embedRange, embeddings)
    print(a,b,c)

    r = np.linspace(1,instRange[-1],10)
    ax.plot(r, f(r, a, b, c), ms=0, color=colour, ls="--", label="Projected")

fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(8,3), width_ratios=(4,6))
ax1.set(ylabel="Qubits")

plotEmbedding("haydn-op1", 4, ax1, cset[0], "o", "Haydn Op. 1")
plotEmbedding("beethoven-op67", 4, ax2, cset[0], "o", "Beethoven Op. 67", extend=2)

lines, labels = plt.gca().get_legend_handles_labels()

#leg = fig.legend([lines[0],(lines[1],lines[2])], [labels[0],labels[1]],loc="upper center", bbox_to_anchor=(0.45,.8))

fig.tight_layout()

plt.savefig(figPath+"embedding.svg")

### Pegasus architecture

In [ ]:
P = dnx.pegasus_graph(3)
plt.figure(figsize=(4,4))
dnx.draw_pegasus(P, with_labels=False, node_size = 15, width=.2, crosses=True, node_color=cset[0], edge_color="k")
plt.savefig("..\Figures\pegasus.svg")

### Toy score

In [ ]:
# Toy example score
f1, f2, v1, v2, b1 = cset[1:6]

toy = stream.Score()

flute = converter.parse("tinyNotation: 4/4 r8 d' d' d' g' a' b' a' g'2 r")
for el in flute.flatten().getElementsByOffset(0.5,3.5):
    el.style.color = f1
for el in flute.flatten().getElementsByOffset(4,5):
    el.style.color = f2
flute.insert(0,instrument.Flute())
toy.insert(0,flute)

violin = converter.parse("tinyNotation: 4/4 r2 b4 a4 g4 g8 d g2")
for el in violin.flatten().getElementsByOffset(2,3):
    el.style.color = v1
for el in violin.flatten().getElementsByOffset(4,7):
    el.style.color = v2
violin.insert(0,instrument.Violin())
toy.insert(0,violin)

bassoon = converter.parse("tinyNotation: 4/4 GG1~ GG1")
bassoon.insert(0,instrument.Bassoon())
for el in bassoon.flatten().notes:
    el.style.color = b1
toy.insert(0,bassoon)

toy.show()
#toy.write(fmt="musicxml", fp="../Figures/toy-score")

In [ ]:
# Toy selected score
toy = stream.Score()

arrangement = converter.parse("tinyNotation: 4/4 r8 d' d' d' g' a' b' a' g4 g8 d g2")
for el in arrangement.flatten().getElementsByOffset(0.5,3.5):
    el.style.color = f1
for el in arrangement.flatten().getElementsByOffset(4,7):
    el.style.color = v2
toy.insert(0,arrangement)

toy.show()
toy.write(fmt="musicxml", fp="../Figures/toy-arrangement")

In [ ]:
# Toy example graph

edges = [("F1","V1"), ("V1","B1"), ("F1","B1"), ("F2","V2"), ("V2","B1"), ("F2","B1")]

F = nx.Graph()
F.add_edges_from(edges)

args = {
    "pos": {"F1": (0,1), "V1": (0,0), "F2": (1,1), "V2": (1,0), "B1": (.5,.5)},
    "node_color": [f1, v1, b1, f2, v2],
    "edge_color": "k",
    "font_color": "w",
    "font_size": 10,
    "width": 1.5,
    "node_size": 1200,
    "hide_ticks": True
}

plt.figure(figsize=(4,2))
selected = ["F1", "V2"]
nx.draw_networkx_nodes(F, pos=args["pos"], edgecolors="k", linewidths=[8,0,0,0,8], node_size=args["node_size"], margins=(.1,.3))
nx.draw_networkx(F, **args)
plt.axis("off")
plt.tight_layout()
plt.savefig("../Figures/toy-graph.svg")

### Appendix

In [ ]:
import ast

sampleset = pd.read_csv(dataPath(IDENTIFIER, 4) + "sampleset.csv")
sample = ast.literal_eval(sampleset.sort_values("energy")["sample"].iloc[1])

In [ ]:
from samples import overlaps, duplicates, extractChosen
from scores import saveToMidi

def plotSampleGraph(sample: dict, G: nx.Graph, instruments: dict) -> None:
    '''
    Plots a sample as a graph.
    '''

    chosen = extractChosen(sample)
    nx.set_node_attributes(G, "None", "assignment")

    for index in chosen:
        G.nodes[f"{index[0]}_{index[1]}"]["assignment"] = chosen[index]

    plt.figure(figsize=(8,8))
    #pos = nx.multipartite_layout(G, "assignment", "horizontal", 2)
    pos = nx.spring_layout(G, k=.3, seed=7)

    colours = [instruments[a]["colour"] if a != "None" else "black" for (_, a) in G.nodes(data="assignment")]
    entropies = np.array([e for (_, e) in G.nodes(data="entropy")])
    edgeWeights = np.array([d["weight"] for _, _, d in G.edges.data()])

    nx.draw_networkx_nodes(G, pos, node_color=colours, node_size=10)
    nx.draw_networkx_edges(G, pos, width=.2)
    plt.axis("off")

IDENTIFIER = "haydn-op1"
NUM = 3

G = nx.read_graphml(picklePath(IDENTIFIER) + "graph.graphml")
sample = json.load(open(dataPath(IDENTIFIER, NUM) + "sample.json"))
instruments = json.load(open(numberPath(IDENTIFIER, NUM) + "instruments.json"))
score = converter.parse(picklePath(IDENTIFIER) + "score.musicxml")
phrases = pd.read_csv(picklePath(IDENTIFIER) + "phrases.csv", index_col=[0,1])

print(overlaps(sample, G), duplicates(sample, G))

plotSampleGraph(sample, G, instruments)
#plt.savefig(figPath+f"beethoven-graph.svg")

arrangement = composeArrangement(sample, score, phrases, instruments)
arrangement.show()
#arrangement.write("musicxml", "../Examples/haydn-arr")

In [ ]:
arrangement = converter.parse("../Examples/" + "haydn-arr.musicxml")
arrangement.insert(0, metadata.Metadata())
arrangement.metadata.movementName = "Quartet No. 1 in B♭ major"
arrangement.metadata.composer = "Joseph Haydn\narr. by Lucas Kirby"
arrangement.write("musicxml.pdf", "../Examples/haydn-arr")

In [ ]:
arrangement = converter.parse("../Examples/" + "haydn-arr.musicxml")
arrangement.write("musicxml.pdf", "../Examples/haydn-arr")
saveToMidi(arrangement, "../Examples/haydn-arr.mid")